In [ ]:
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.1.3 --quiet
import pytorch_lightning as pl

     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 686kB 36.5MB/s 
     |████████████████████████████████| 645kB 37.9MB/s 
     |████████████████████████████████| 112kB 37.0MB/s 
     |████████████████████████████████| 829kB 27.6MB/s 
     |████████████████████████████████| 1.3MB 48.1MB/s 
     |████████████████████████████████| 296kB 45.5MB/s 
     |████████████████████████████████| 143kB 55.4MB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.


In [ ]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint


In [ ]:
print("torch version:",torch.__version__)
print("pytorch ligthening version:",pl.__version__)

torch version: 1.7.1
pytorch ligthening version: 1.1.3


In [ ]:
inputs = [Variable(torch.Tensor([0, 0])),
           Variable(torch.Tensor([0, 1])),
           Variable(torch.Tensor([1, 0])),
           Variable(torch.Tensor([1, 1]))]

In [ ]:
targets = [Variable(torch.Tensor([0])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([0]))]

In [ ]:
data_inputs_targets = list(zip(inputs, targets))
data_inputs_targets

[(tensor([0., 0.]), tensor([0.])),
 (tensor([0., 1.]), tensor([1.])),
 (tensor([1., 0.]), tensor([1.])),
 (tensor([1., 1.]), tensor([0.]))]

In [ ]:
class XOR(pl.LightningModule):
  def __init__(self):
    super(XOR,self).__init__()
    
    self.input_layer = nn.Linear(2, 4)
    self.output_layer = nn.Linear(4,1)

    self.sigmoid = nn.Sigmoid()

    self.loss = nn.MSELoss()

  def forward(self, input):
    x = self.input_layer(input)
    x = self.sigmoid(x)
    output = self.output_layer(x)
    return output

  def configure_optimizers(self):
    params = self.parameters()
    optimizer = optim.Adam(params=params, lr = 0.01)
    return optimizer

  def training_step(self, batch, batch_idx):
    inputs, targets = batch
    outputs = self(inputs) 
    loss = self.loss(outputs, targets)
    return loss 



In [ ]:
checkpoint_callback = ModelCheckpoint()
model = XOR()

trainer = pl.Trainer(max_epochs=500, callbacks=[checkpoint_callback], progress_bar_refresh_rate=30)
trainer.fit(model, train_dataloader=data_inputs_targets)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params


1

In [ ]:
ls lightning_logs/

version_0/  version_1/


In [ ]:
ls lightning_logs/*/

lightning_logs/version_0/:
checkpoints/  events.out.tfevents.1615662604.a914eb882a31.62.0  hparams.yaml

lightning_logs/version_1/:
checkpoints/  events.out.tfevents.1615662832.a914eb882a31.62.1  hparams.yaml

lightning_logs/version_2/:
checkpoints/  events.out.tfevents.1615662838.a914eb882a31.62.2  hparams.yaml


In [ ]:
print(checkpoint_callback.best_model_path)
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
test = torch.utils.data.DataLoader(inputs, batch_size=1)
for val in inputs:
  _ = train_model(val)
  print([int(val[0]),int(val[1])], int(_.round()))

/content/lightning_logs/version_1/checkpoints/epoch=499-step=1999.ckpt
[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 0


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  warnings.warn(*args, **kwargs)


1